In [ ]:
from huggingface_hub import login

login(
  token="", # ADD YOUR TOKEN HERE
  add_to_git_credential=True
)

Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 11.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 55.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 56.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver d

In [ ]:
pip install trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.6/316.6 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.1/108.1 kB 9.8 MB/s eta 0:00:00


In [ ]:
pip install qwen-vl-utils

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.1/33.1 MB 63.8 MB/s eta 0:00:00


In [ ]:
# note the image is not provided in the prompt its included as part of the "processor"
prompt= """Create a Short Product description based on the provided ##PRODUCT NAME## and ##CATEGORY## and image.
Only return description. The description should be SEO optimized and for a better mobile search experience.

##PRODUCT NAME##: {product_name}
##CATEGORY##: {category}"""

system_message = "You are an expert product description writer for Amazon."

In [ ]:
from datasets import load_dataset

# Convert dataset to OAI messages
def format_data(sample):
    return {"messages": [
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "text",
                            "text": prompt.format(product_name=sample["Product Name"], category=sample["Category"]),
                        },{
                            "type": "image",
                            "image": sample["image"],
                        }
                    ],
                },
                {
                    "role": "assistant",
                    "content": [{"type": "text", "text": sample["description"]}],
                },
            ],
        }

# Load dataset from the hub
dataset_id = "philschmid/amazon-product-descriptions-vlm"
dataset = load_dataset("philschmid/amazon-product-descriptions-vlm", split="train")

# Convert dataset to OAI messages
# need to use list comprehension to keep Pil.Image type, .mape convert image to bytes
dataset = [format_data(sample) for sample in dataset]

print(dataset[345]["messages"])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


[{'role': 'user', 'content': [{'type': 'text', 'text': 'Create a Short Product description based on the provided ##PRODUCT NAME## and ##CATEGORY## and image.\nOnly return description. The description should be SEO optimized and for a better mobile search experience.\n\n##PRODUCT NAME##: MasterPieces Tribal Spirit Jigsaw Puzzle, The Chiefs, Featuring American Indian Tribe Traditions & Ceremonies, 1000 Pieces\n##CATEGORY##: Toys & Games | Puzzles | Jigsaw Puzzles'}, {'type': 'image', 'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=500x500 at 0x7F7951C3EE30>}]}, {'role': 'assistant', 'content': [{'type': 'text', 'text': 'Challenge yourself with this 1000-piece MasterPieces Tribal Spirit jigsaw puzzle!  Depicting the rich traditions and ceremonies of American Indian tribes, "The Chiefs" offers a stunning, culturally significant image perfect for puzzle enthusiasts.  High-quality pieces guarantee a satisfying solve.'}]}]


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, set_seed, Trainer, TrainingArguments, BitsAndBytesConfig, \
    DataCollatorForLanguageModeling, Trainer, TrainingArguments, GenerationConfig

In [ ]:
import torch

def create_bnb_config():
    bnb_4bit_compute_dtype = "float16"
    compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=compute_dtype,
    )
    return bnb_config

In [ ]:
quant_config = create_bnb_config()

In [ ]:
import requests
import torch
from PIL import Image
from transformers import MllamaForConditionalGeneration, AutoProcessor

model_id = "meta-llama/Llama-3.2-11B-Vision-Instruct"

model = MllamaForConditionalGeneration.from_pretrained(
    model_id,
    quantization_config=quant_config,
    device_map="auto",
)


ValueError: Some modules are dispatched on the CPU or the disk. Make sure you have enough GPU RAM to fit the quantized model. If you want to dispatch the model on the CPU or the disk while keeping these modules in 32-bit, you need to set `load_in_8bit_fp32_cpu_offload=True` and pass a custom `device_map` to `from_pretrained`. Check https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu for more details. 

In [ ]:
from PIL import Image
import io
import base64

# Replace with the path to your image
image_path = 'image.png'

# Open the image
with Image.open(image_path) as img:
    # Convert the image to RGB mode (in case it's not already)
    img = img.convert('RGB')

    # Create a byte stream
    buffered = io.BytesIO()

    # Save the image to the byte stream in JPEG format
    img.save(buffered, format="JPEG")

    # Encode the bytes to base64
    img_str = base64.b64encode(buffered.getvalue()).decode()

print(f"Base64 string of the image (first 100 characters):\n{img_str[:100]}...")

Base64 string of the image (first 100 characters):
/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBwcJCQgKDBQNDAsLDBkSEw8UHRofHh0aHBwgJC4nICIsIxwcKDcpLDAx...


In [ ]:
processor = AutoProcessor.from_pretrained(model_id)

url = "https://huggingface.co/datasets/huggingface/documentation-images/resolve/0052a70beed5bf71b92610a43a52df6d286cd5f3/diffusers/rabbit.jpg"
image = Image.open(requests.get(url, stream=True).raw)

messages = [
    {"role": "user", "content": [
        {"type": "image"},
        {"type": "text", "text": "What is the image about ? "}
    ]}
]
input_text = processor.apply_chat_template(messages, add_generation_prompt=True)
inputs = processor(
    image,
    input_text,
    add_special_tokens=False,
    return_tensors="pt"
).to(model.device)

output = model.generate(**inputs, max_new_tokens=30)
print(processor.decode(output[0]))

<|begin_of_text|><|start_header_id|>user<|end_header_id|>

<|image|>What is the image about? <|eot_id|><|start_header_id|>assistant<|end_header_id|>

The image depicts a charming scene of a rabbit standing on a dirt path, surrounded by a picturesque landscape. The rabbit is dressed in a blue jacket and


In [ ]:
processor = AutoProcessor.from_pretrained(model_id)

url = "https://huggingface.co/datasets/huggingface/documentation-images/resolve/0052a70beed5bf71b92610a43a52df6d286cd5f3/diffusers/rabbit.jpg"
image = Image.open(requests.get(url, stream=True).raw)

messages = [
    {"role": "user", "content": [
        {"type": "image"},
        {"type": "text", "text": "What is the image about ? "}
    ]}
]
input_text = processor.apply_chat_template(messages, add_generation_prompt=True)
inputs = processor(
    img,
    input_text,
    add_special_tokens=False,
    return_tensors="pt"
).to(model.device)

output = model.generate(**inputs, max_new_tokens=300)
print(processor.decode(output[0]))

<|begin_of_text|><|start_header_id|>user<|end_header_id|>

<|image|>What is the image about? <|eot_id|><|start_header_id|>assistant<|end_header_id|>

The image shows a screenshot of the Salesforce platform, with a focus on the "Actions" and "Conversation Preview" sections. The "Actions" section is located on the left side of the screen and features a list of available actions, including "This Copilot's Actions," "Copilot Action Library," "7 items - Sorted by Copilot Action Label(asc)," "Copilot Action Label," "Identify Object by Name," "Summarize Record," "Identify Record by Name," "Query Records," "Query Records with Aggregate," "Draft or Revise Sales Email," and "Answer Questions with Knowledge." The "Conversation Preview" section is located on the right side of the screen and displays a conversation between two users, with the user's name, email address, and message displayed. The conversation appears to be a discussion about a sales opportunity, with the user providing information 

In [ ]:
from PIL import Image
import io
import base64

# Replace with the path to your image
image_path = 'file-5d27ffbe9f837.jpg'

# Open the image
with Image.open(image_path) as img:
    # Convert the image to RGB mode (in case it's not already)
    img = img.convert('RGB')

    # Create a byte stream
    buffered = io.BytesIO()

    # Save the image to the byte stream in JPEG format
    img.save(buffered, format="JPEG")

    # Encode the bytes to base64
    img_str = base64.b64encode(buffered.getvalue()).decode()

print(f"Base64 string of the image (first 100 characters):\n{img_str[:100]}...")

Base64 string of the image (first 100 characters):
/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBwcJCQgKDBQNDAsLDBkSEw8UHRofHh0aHBwgJC4nICIsIxwcKDcpLDAx...


In [ ]:
processor = AutoProcessor.from_pretrained(model_id)

url = "https://huggingface.co/datasets/huggingface/documentation-images/resolve/0052a70beed5bf71b92610a43a52df6d286cd5f3/diffusers/rabbit.jpg"
image = Image.open(requests.get(url, stream=True).raw)

messages = [
    {"role": "user", "content": [
        {"type": "image"},
        {"type": "text", "text": "Extract all the information from the image in json "}
    ]}
]
input_text = processor.apply_chat_template(messages, add_generation_prompt=True)
inputs = processor(
    img,
    input_text,
    add_special_tokens=False,
    return_tensors="pt"
).to(model.device)

output = model.generate(**inputs, max_new_tokens=300)
print(processor.decode(output[0]))

<|begin_of_text|><|start_header_id|>user<|end_header_id|>

<|image|>Extract all the information from the image in json <|eot_id|><|start_header_id|>assistant<|end_header_id|>

"{
    \"Income from tourism sector\": [
        {
            \"Year\": \"1982\",
            \"Value\": \"0.4\"
        },
        {
            \"Year\": \"1993\",
            \"Value\": \"1.8\"
        },
        {
            \"Year\": \"1995\",
            \"Value\": \"2.1\"
        },
        {
            \"Year\": \"1997\",
            \"Value\": \"2.9\"
        },
        {
            \"Year\": \"2000\",
            \"Value\": \"4.4\"
        },
        {
            \"Year\": \"2001\",
            \"Value\": \"3.9\"
        },
        {
            \"Year\": \"2002\",
            \"Value\": \"3.9\"
        },
        {
            \"Year\": \"2003\",
            \"Value\": \"1.6\"
        }
    ],
    \"Income from construction sector\": [
        {
            \"Year\": \"1982\",
            \"Value\

In [ ]:
## Finetuning

In [ ]:
dataset[345]["messages"]

[{'role': 'user',
  'content': [{'type': 'text',
    'text': 'Create a Short Product description based on the provided ##PRODUCT NAME## and ##CATEGORY## and image.\nOnly return description. The description should be SEO optimized and for a better mobile search experience.\n\n##PRODUCT NAME##: MasterPieces Tribal Spirit Jigsaw Puzzle, The Chiefs, Featuring American Indian Tribe Traditions & Ceremonies, 1000 Pieces\n##CATEGORY##: Toys & Games | Puzzles | Jigsaw Puzzles'},
   {'type': 'image',
    'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=500x500>}]},
 {'role': 'assistant',
  'content': [{'type': 'text',
    'text': 'Challenge yourself with this 1000-piece MasterPieces Tribal Spirit jigsaw puzzle!  Depicting the rich traditions and ceremonies of American Indian tribes, "The Chiefs" offers a stunning, culturally significant image perfect for puzzle enthusiasts.  High-quality pieces guarantee a satisfying solve.'}]}]

In [ ]:
import torch
from transformers import AutoModelForVision2Seq, AutoProcessor, BitsAndBytesConfig

# Hugging Face model id
model_id = "meta-llama/Llama-3.2-11B-Vision-Instruct"

# BitsAndBytesConfig int-4 config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16
)

# Load model and tokenizer
model = AutoModelForVision2Seq.from_pretrained(
    model_id,
    device_map="auto",
    # attn_implementation="flash_attention_2", # not supported for training
    torch_dtype=torch.bfloat16,
    quantization_config=bnb_config
)
processor = AutoProcessor.from_pretrained(model_id)

text = processor.apply_chat_template(
    dataset[2]["messages"], tokenize=False, add_generation_prompt=False
)

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
text

'<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\nCreate a Short Product description based on the provided ##PRODUCT NAME## and ##CATEGORY## and image.\nOnly return description. The description should be SEO optimized and for a better mobile search experience.\n\n##PRODUCT NAME##: Barbie Fashionistas Doll Wear Your Heart\n##CATEGORY##: Toys & Games | Dolls & Accessories | Dolls<|image|><|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nExpress your style with Barbie Fashionistas Doll Wear Your Heart! This fashionable doll boasts a unique outfit and accessories, perfect for imaginative play.  A great gift for kids aged 3+.  Collect them all! #Barbie #Fashionistas #Doll #Toys #GirlsToys #FashionDoll #Play<|eot_id|>'

In [ ]:
from peft import LoraConfig

# LoRA config based on QLoRA paper & Sebastian Raschka experiment
peft_config = LoraConfig(
        lora_alpha=16,
        lora_dropout=0.05,
        r=8,
        bias="none",
        target_modules=["q_proj", "v_proj"],
        task_type="CAUSAL_LM",
)

In [ ]:
from trl import SFTConfig
from transformers import Qwen2VLProcessor
from qwen_vl_utils import process_vision_info

args = SFTConfig(
    output_dir="finetuned-vision-llama", # directory to save and repository id
    num_train_epochs=3,                     # number of training epochs
    per_device_train_batch_size=4,          # batch size per device during training
    gradient_accumulation_steps=8,          # number of steps before performing a backward/update pass
    gradient_checkpointing=True,            # use gradient checkpointing to save memory
    optim="adamw_torch_fused",              # use fused adamw optimizer
    logging_steps=5,                       # log every 10 steps
    save_strategy="epoch",                  # save checkpoint every epoch
    learning_rate=2e-4,                     # learning rate, based on QLoRA paper
    bf16=True,                              # use bfloat16 precision
    tf32=True,                              # use tf32 precision
    max_grad_norm=0.3,                      # max gradient norm based on QLoRA paper
    warmup_ratio=0.03,                      # warmup ratio based on QLoRA paper
    lr_scheduler_type="constant",           # use constant learning rate scheduler
    push_to_hub=True,                       # push model to hub
    report_to="tensorboard",                # report metrics to tensorboard
    gradient_checkpointing_kwargs = {"use_reentrant": False}, # use reentrant checkpointing
    dataset_text_field="", # need a dummy field for collator
    dataset_kwargs = {"skip_prepare_dataset": True} # important for collator
)
args.remove_unused_columns=False


In [ ]:
from qwen_vl_utils import process_vision_info

# Define a custom collator function
def collate_fn(examples):
    print(examples)
    texts = [processor.apply_chat_template(example["messages"], tokenize=False) for example in examples]
    image_inputs = [process_vision_info(example["messages"])[0] for example in examples]

    batch = processor(text=texts, images=image_inputs, return_tensors="pt", padding=True)
    labels = batch["input_ids"].clone()
    labels[labels == processor.tokenizer.pad_token_id] = -100

    # Mask image tokens in the labels
    image_tokens = [processor.tokenizer.convert_tokens_to_ids(processor.image_token)]
    for image_token_id in image_tokens:
        labels[labels == image_token_id] = -100
    batch["labels"] = labels

    return batch

In [ ]:
dataset[0]["messages"][0]

{'role': 'user',
 'content': [{'type': 'text',
   'text': "Create a Short Product description based on the provided ##PRODUCT NAME## and ##CATEGORY## and image.\nOnly return description. The description should be SEO optimized and for a better mobile search experience.\n\n##PRODUCT NAME##: Kurio Glow Smartwatch for Kids with Bluetooth, Apps, Camera & Games, Blue\n##CATEGORY##: Toys & Games | Kids' Electronics | Electronic Learning Toys"},
  {'type': 'image',
   'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=500x500>}]}

In [ ]:
collate_fn(dataset[0:3])['input_ids']

[{'messages': [{'role': 'user', 'content': [{'type': 'text', 'text': "Create a Short Product description based on the provided ##PRODUCT NAME## and ##CATEGORY## and image.\nOnly return description. The description should be SEO optimized and for a better mobile search experience.\n\n##PRODUCT NAME##: Kurio Glow Smartwatch for Kids with Bluetooth, Apps, Camera & Games, Blue\n##CATEGORY##: Toys & Games | Kids' Electronics | Electronic Learning Toys"}, {'type': 'image', 'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=500x500 at 0x7F7978C1B820>}]}, {'role': 'assistant', 'content': [{'type': 'text', 'text': "Kurio Glow Smartwatch: Fun, Safe & Educational!  This kids' smartwatch boasts Bluetooth connectivity, built-in apps & games, and a camera – all in a vibrant blue design. Perfect for learning & play!  #kidssmartwatch #kidselectronics #educationaltoys #kurioglow"}]}]}, {'messages': [{'role': 'user', 'content': [{'type': 'text', 'text': 'Create a Short Product description b

tensor([[128000, 128000, 128006,    882, 128007,    271,   4110,    264,  10928,
           5761,   4096,   3196,    389,    279,   3984,   7860,  52531,  19668,
            567,    323,   7860,  94127,    567,    323,   2217,    627,   7456,
            471,   4096,     13,    578,   4096,   1288,    387,  26029,  34440,
            323,    369,    264,   2731,   6505,   2778,   3217,    382,    567,
          52531,  19668,    567,     25,  32375,    822,  88448,  16147,  14658,
            369,  23338,    449,  24783,     11,  34120,     11,  14669,    612,
          11871,     11,   8868,    198,    567,  94127,    567,     25,  49481,
            612,  11871,    765,  23338,      6,  38784,    765,  35269,  21579,
          49481, 128256, 128009, 128006,  78191, 128007,    271,     42,    324,
            822,  88448,  16147,  14658,     25,  16457,     11,  23088,    612,
          46945,      0,    220,   1115,   6980,      6,   7941,  14658,  38119,
          24783,  31357,    

In [ ]:
from trl import SFTTrainer

# Trainer setup
trainer = SFTTrainer(
    model=model,
    args=args,
    train_dataset=dataset,
    data_collator=collate_fn,
    tokenizer=processor.tokenizer,
    peft_config=peft_config
)

# Start training
trainer.train()

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:292: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:401: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  super().__init__(


[{'messages': [{'role': 'user', 'content': [{'type': 'text', 'text': "Create a Short Product description based on the provided ##PRODUCT NAME## and ##CATEGORY## and image.\nOnly return description. The description should be SEO optimized and for a better mobile search experience.\n\n##PRODUCT NAME##: LEISURE ARTS A Dishcloth A Month\n##CATEGORY##: Home & Kitchen | Home Décor | Kids' Room Décor"}, {'type': 'image', 'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=500x500 at 0x7F79AFF532B0>}]}, {'role': 'assistant', 'content': [{'type': 'text', 'text': 'Adorable & functional dishcloths!  LEISURE ARTS\' "A Dishcloth A Month" collection adds charming style to your kitchen or kids\' room.  Perfect for everyday use or as unique home décor.  Shop now!'}]}]}, {'messages': [{'role': 'user', 'content': [{'type': 'text', 'text': 'Create a Short Product description based on the provided ##PRODUCT NAME## and ##CATEGORY## and image.\nOnly return description. The description should be 

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:1399: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with device_autocast_ctx, torch.cpu.amp.autocast(**cpu_autocast_kwargs), recompute_context:  # type: ignore[attr-defined]


[{'messages': [{'role': 'user', 'content': [{'type': 'text', 'text': 'Create a Short Product description based on the provided ##PRODUCT NAME## and ##CATEGORY## and image.\nOnly return description. The description should be SEO optimized and for a better mobile search experience.\n\n##PRODUCT NAME##: Barbie Dreamhouse Adventures Stacie Doll, Approx. 9-inch\n##CATEGORY##: Toys & Games | Dolls & Accessories | Dolls'}, {'type': 'image', 'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=167x500 at 0x7F7951549CC0>}]}, {'role': 'assistant', 'content': [{'type': 'text', 'text': 'Bring home the fun with Barbie Dreamhouse Adventures Stacie Doll! This approximately 9-inch doll is perfect for imaginative play.  A great gift for kids who love Barbie!'}]}]}, {'messages': [{'role': 'user', 'content': [{'type': 'text', 'text': 'Create a Short Product description based on the provided ##PRODUCT NAME## and ##CATEGORY## and image.\nOnly return description. The description should be SEO opt

Step,Training Loss


[{'messages': [{'role': 'user', 'content': [{'type': 'text', 'text': 'Create a Short Product description based on the provided ##PRODUCT NAME## and ##CATEGORY## and image.\nOnly return description. The description should be SEO optimized and for a better mobile search experience.\n\n##PRODUCT NAME##: Steiff Happy Farm Skittles Bowling Set\n##CATEGORY##: Toys & Games | Stuffed Animals & Plush Toys | Stuffed Animals & Teddy Bears'}, {'type': 'image', 'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=500x500 at 0x7F7951A9C4C0>}]}, {'role': 'assistant', 'content': [{'type': 'text', 'text': 'Adorable Steiff Happy Farm Skittles Bowling Set!  Ten charming plush animals become fun bowling pins for hours of playful fun. Perfect gift for toddlers and preschoolers.  Develops hand-eye coordination and fine motor skills.  High-quality Steiff craftsmanship. Shop now!'}]}]}, {'messages': [{'role': 'user', 'content': [{'type': 'text', 'text': 'Create a Short Product description based on 

KeyboardInterrupt: 